Good luck with the continuation of the project!
-Lukas

TODO:
- James Msc:
 + gini index
 + RUST

 - tests
   + read_frame_score test
 - how much data is enough to be representative
 - R riboseq data generator (by ORFIK designer, and Amanda MOK)

 - chunking data (pysam)
 - multiprocessing
 - annotation merge is too specific
 - generate both ends ligation bias, variable plot names?
 - ligation bias, color underrepresented groups differently from overrepresented
 - fix read frame score placement for pdf
 - check list (awesome riboseq)
 - SCORING: each module needs scores
   + ligation bias = [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)

In [2]:
from modules import *
from metrics import *
#from plots import *
import pysam
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import gffpandas.gffpandas as gffpd
import pandas as pd
import numpy as np
import timeit
import yaml
# from file_parser import parse_bam
# from bam_processing import *
import base64
import sys
import time

In [2]:
with open("../RiboMetric/config.yml", "r") as ymlfile:
        config = yaml.load(ymlfile, Loader=yaml.Loader)
# config

In [3]:
# for new read_df, use parse_to_csv.py
# read_df_pre = pd.read_csv("/home/lukas/projects/RiboMetric/test_files/sample_read_df_pre_1000000.csv")
read_df_pre = pd.read_csv("/home/lukas/projects/RiboMetric/test_files/sample_read_df_pre.csv")
read_df_pre["reference_name"] = read_df_pre["reference_name"].astype("category")
annotation_df = pd.read_csv("/home/lukas/projects/RiboMetric/sample_data/gencode.v25.annotation_RiboMetric.tsv",delimiter="\t")

In [4]:
read_df = read_df_pre.loc[read_df_pre.index.repeat(read_df_pre['count'])].reset_index(drop=True)
read_df = a_site_calculation(read_df)
annotated_read_df = annotate_reads(read_df, annotation_df)
annotated_read_df = assign_mRNA_category(annotated_read_df)
cds_read_df = read_df_to_cds_read_df(annotated_read_df)

---------------

In [4]:
read_df_pre = pd.read_csv("../tests/test_data/test.csv")
read_df = read_df_pre.loc[
    read_df_pre.index.repeat(read_df_pre["count"])
].reset_index(drop=True)
read_df = a_site_calculation(read_df)

---------------

In [116]:
def extract_transcript_id(attr_str):
    for attr in attr_str.split(";"):
        # Ensembl GFF3 support
        if attr.startswith("Parent=transcript:") \
                or attr.startswith("ID=transcript:"):
            return attr.split(":")[1]
        # Gencode GFF3 support
        elif attr.startswith("transcript_id="):
            return attr.split("=")[1]
        # Ensembl GTF support
        elif attr.startswith(" transcript_id "):
            return attr.split(" ")[2].replace('"', "")
    return np.nan

In [117]:
def gff_df_to_cds_df(
        gff_df: pd.DataFrame,
        transcript_list: list
        ) -> pd.DataFrame:
    """
    Subset the gff dataframe to only include the CDS features
    with tx coordinates for a specific list of transcripts.

    Inputs:
        gff_df: Dataframe containing the gff information
        transcript_list: List of transcripts to subset

    Outputs:
        cds_df: Dataframe containing the CDS information
                columns: transcript_id, cds_start, cds_end
    """
    # Extract transcript ID from "attributes" column using regular expression
    rows = {
        "transcript_id": [],
        "cds_start": [],
        "cds_end": [],
        "transcript_length": [],
        "genomic_cds_starts": [],
        "genomic_cds_ends": [],
    }

    # Sort GFF DataFrame by transcript ID
    gff_df = gff_df.sort_values("transcript_id")

    # subset GFF DataFrame to only include transcripts in the transcript_list
    gff_df = gff_df[gff_df["transcript_id"].isin(transcript_list)]

    counter = 0
    for group_name, group_df in gff_df.groupby("transcript_id"):
        counter += 1
        if counter % 100 == 0:
            prop = counter / len(transcript_list)
            print(f"Processing transcript ({prop*100}%)", end="\r")
        if group_name in transcript_list:
            transcript_start = group_df["start"].min()

            cds_df_tx = group_df[group_df["type"] == "CDS"]
            cds_start_end_tuple_list = sorted(
                zip(cds_df_tx["start"], cds_df_tx["end"])
                )

            cds_tx_start = cds_start_end_tuple_list[0][0] - transcript_start
            cds_tx_end = cds_start_end_tuple_list[-1][1] - transcript_start

            for cds in cds_start_end_tuple_list:
                cds_length = cds[1] - cds[0]
                cds_tx_end += cds_length

            genomic_cds_starts = ",".join(
                [str(x[0]) for x in cds_start_end_tuple_list]
                )

            genomic_cds_ends = ",".join(
                [str(x[1]) for x in cds_start_end_tuple_list]
                )

            rows["transcript_id"].append(group_name)
            rows["cds_start"].append(cds_tx_start)
            rows["cds_end"].append(cds_tx_end)
            rows["transcript_length"].append(cds_tx_end - cds_tx_start)
            rows["genomic_cds_starts"].append(genomic_cds_starts)
            rows["genomic_cds_ends"].append(genomic_cds_ends)

    return pd.DataFrame(rows)


In [276]:
def split_gff_file(input_file, output_prefix, num_files):
    with open(input_file, 'r') as f:
        gff_lines = f.readlines()

    num_lines = len(gff_lines)
    lines_per_split = num_lines // num_files

    split_indices = [0]
    line_number = 0
    for i in range(num_lines):
        line_number += 1
        line = gff_lines[i]
        if line_number >= lines_per_split:
            if line.startswith("#"):
                continue
            if line.split('\t')[2] == "gene":
                split_indices.append(i)
                line_number = 0

    split_indices.append(num_lines)
    print(split_indices)

    for i in range(num_files):
        start_index = split_indices[i]
        end_index = min(split_indices[i + 1], num_lines)
        file_lines = gff_lines[start_index:end_index]

        output_file = f"{output_prefix}_{i + 1}.gff"
        with open(output_file, 'w') as f_out:
            f_out.writelines(file_lines)

        print(f"Created file: {output_file}")

In [277]:
split_gff_file("/home/lukas/projects/RiboMetric/test_files/Saccharomyces_cerevisiae.R64-1-1.109.gff3","tempgff", 5)

[0, 7171, 14353, 21527, 28703, 35862]
Created file: tempgff_1.gff
Created file: tempgff_2.gff
Created file: tempgff_3.gff
Created file: tempgff_4.gff
Created file: tempgff_5.gff


In [193]:
num_lines = sum(1 for _ in open("/home/lukas/projects/RiboMetric/test_files/gencode.v43.annotation.gff3"))
# num_lines = sum(1 for _ in open(gff_path))
num_lines

3422781

In [186]:
gff = gffpd.read_gff3('/home/lukas/projects/RiboMetric/test_files/Saccharomyces_cerevisiae.R64-1-1.109.gff3')
print(type(gff))
gff.df.loc[:, "transcript_id"] = gff.df["attributes"].apply(
    extract_transcript_id)
gff.df = gff.df.sort_values("transcript_id")
cds = gff.filter_feature_of_type(['CDS'])
num_transcripts = None
coding_tx_ids = cds.df["transcript_id"].unique()[:num_transcripts]

<class 'gffpandas.gffpandas.Gff3DataFrame'>


In [185]:
gff_ver = gff_df_to_cds_df(gff.df, coding_tx_ids)

In [45]:
rows = {
    "transcript_id": [],
    "cds_start": [],
    "cds_end": [],
    "transcript_length": [],
    "genomic_cds_starts": [],
    "genomic_cds_ends": [],
}

# Sort GFF DataFrame by transcript ID
cds.df = cds.df.sort_values("transcript_id")

# subset GFF DataFrame to only include transcripts in the transcript_list
cds.df = cds.df[cds.df["transcript_id"].isin(transcript_list)]

counter = 0
for group_name, group_df in gff_df.groupby("transcript_id"):
    counter += 1
    if counter % 100 == 0:
        prop = counter / len(transcript_list)
        print(f"Processing transcript ({round(prop*100,3)}%)", end="\r")

    transcript_start = group_df["start"].min()

    cds_df_tx = group_df[group_df["type"] == "CDS"]
    cds_start_end_tuple_list = sorted(
        zip(cds_df_tx["start"], cds_df_tx["end"])
        )

    cds_tx_start = cds_start_end_tuple_list[0][0] - transcript_start
    cds_tx_end = cds_start_end_tuple_list[-1][1] - transcript_start

    for cds in cds_start_end_tuple_list:
        cds_length = cds[1] - cds[0]
        cds_tx_end += cds_length

    genomic_cds_starts = ",".join(
        [str(x[0]) for x in cds_start_end_tuple_list]
        )

    genomic_cds_ends = ",".join(
        [str(x[1]) for x in cds_start_end_tuple_list]
        )

    rows["transcript_id"].append(group_name)
    rows["cds_start"].append(cds_tx_start)
    rows["cds_end"].append(cds_tx_end)
    rows["transcript_length"].append(cds_tx_end - cds_tx_start)
    rows["genomic_cds_starts"].append(genomic_cds_starts)
    rows["genomic_cds_ends"].append(genomic_cds_ends)


---------------

In [6]:
read_df["reference_name"].value_counts()

reference_name
ENST00000604646.1|ENSG00000270188.1|OTTHUMG00000184992.1|OTTHUMT00000469412.1|MTRNR2L11-001|MTRNR2L11|1552|protein_coding|                 248842
ENST00000415278.1|ENSG00000228502.1|OTTHUMG00000010710.1|OTTHUMT00000029579.1|EEF1A1P11-001|EEF1A1P11|1389|processed_pseudogene|            42802
ENST00000443590.1|ENSG00000214199.3|OTTHUMG00000153185.1|OTTHUMT00000329979.1|EEF1A1P12-001|EEF1A1P12|1346|processed_pseudogene|            40809
ENST00000609879.1|ENSG00000273213.1|OTTHUMG00000186366.1|OTTHUMT00000473015.1|RP5-998N21.10-001|RP5-998N21.10|700|processed_transcript|     36433
ENST00000414273.1|ENSG00000237973.1|OTTHUMG00000002333.2|OTTHUMT00000006715.2|MTCO1P12-001|MTCO1P12|1543|unprocessed_pseudogene|            35703
                                                                                                                                            ...  
ENST00000410475.1|ENSG00000222407.1|-|-|RNA5SP80-201|RNA5SP80|111|rRNA|                                      

In [16]:
annotated_read_df["reference_name"].value_counts()

reference_name
ENST00000604646.1|ENSG00000270188.1|OTTHUMG00000184992.1|OTTHUMT00000469412.1|MTRNR2L11-001|MTRNR2L11|1552|protein_coding|           248842
ENST00000369155.3|ENSG00000184678.10|OTTHUMG00000012095.1|OTTHUMT00000033455.1|HIST2H2BE-001|HIST2H2BE|2194|protein_coding|           33182
ENST00000369167.2|ENSG00000203814.6|OTTHUMG00000183159.2|OTTHUMT00000033453.3|HIST2H2BF-001|HIST2H2BF|1954|protein_coding|            33021
ENST00000545683.1|ENSG00000203814.6|OTTHUMG00000183159.2|-|HIST2H2BF-201|HIST2H2BF|999|protein_coding|                                32296
ENST00000612061.1|ENSG00000270276.2|OTTHUMG00000188201.2|OTTHUMT00000477867.1|HIST2H4B-002|HIST2H4B|1694|nonsense_mediated_decay|     30400
                                                                                                                                      ...  
ENST00000469565.6|ENSG00000160789.19|OTTHUMG00000013961.5|OTTHUMT00000359807.1|LMNA-025|LMNA|591|processed_transcript|                    0
ENST0

In [15]:
annotated_read_df["a_site"].value_counts()

a_site
1374     60571
375      44297
117      38572
339      35830
525      30324
         ...  
56156        1
56187        1
56228        1
56231        1
64350        1
Name: count, Length: 42358, dtype: int64

In [26]:
def normalize_counts(df: pd.DataFrame) -> pd.Series:
    """
    Calculate the normalised counts depending on the reference for the reads
    """
    reference_counts = annotated_read_df["reference_name"].value_counts()
    return 1 / annotated_read_df['reference_name'].map(reference_counts)

In [29]:
annotated_read_df["TPM"] = normalize_counts(annotated_read_df)

annotated_read_df["metagene_info"] = metagene_distance(
    annotated_read_df, "start"
)

In [30]:
annotated_read_df.head()

,Unnamed: 0,read_length,reference_name,reference_start,first_dinucleotide,last_dinucleotide,count,a_site,transcript_id,cds_start,cds_end,transcript_length,genomic_cds_starts,genomic_cds_ends,mRNA_category,TPM,metagene_info
0,1018,17,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,16,AT,AT,3,31,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,31
1,1018,17,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,16,AT,AT,3,31,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,31
2,1018,17,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,16,AT,AT,3,31,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,31
3,1019,16,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,16,AT,TC,1,31,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,31
4,1020,17,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,17,TT,TA,1,32,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,32


In [47]:
annotated_read_df["readlength_metagene"] = (annotated_read_df["read_length"].astype(str)
                                + "_"
                                + annotated_read_df["metagene_info"].astype(str)
                                ).astype("category")
# annotated_read_df["readlength_metagene"] = annotated_read_df.apply(
#     lambda row: hash((row["read_length"], row["metagene_info"])),
#     axis=1
# )

In [48]:
sum_reciprocal = annotated_read_df.groupby(["read_length", "metagene_info"])["TPM"].sum()

In [50]:
sum_reciprocal.to_dict

<bound method Series.to_dict of read_length  metagene_info
15           -867003          0.012658
             -866903          0.012658
             -866902          0.012658
             -866885          0.012658
             -866853          0.012658
                                ...   
50            96283           0.000128
              98335           0.000116
              99066           0.000126
              99864           0.000127
              103989          0.000116
Name: TPM, Length: 726868, dtype: float64>

In [46]:
annotated_read_df.head()

,Unnamed: 0,read_length,reference_name,reference_start,first_dinucleotide,last_dinucleotide,count,a_site,transcript_id,cds_start,cds_end,transcript_length,genomic_cds_starts,genomic_cds_ends,mRNA_category,TPM,metagene_info,readlength_metagene
0,1018,17,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,16,AT,AT,3,31,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,31,7996716941359164789
1,1018,17,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,16,AT,AT,3,31,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,31,7996716941359164789
2,1018,17,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,16,AT,AT,3,31,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,31,7996716941359164789
3,1019,16,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,16,AT,TC,1,31,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,31,-8012969285610141134
4,1020,17,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,17,TT,TA,1,32,ENST00000335137.3,0,1834,1834,69091,70008,CDS,0.043478,32,-3942386376248388402


In [51]:
# ADDED NORMALIZED COUNTS, expand cell to see

def metagene_profile3(
    annotated_read_df: pd.DataFrame,
    target: str = "both",
    distance_range: list = [-50, 50],
) -> dict:
    """
    Groups the reads by read_length and distance to a target and counts them

    Inputs:
        annotated_read_df: Dataframe containing the read information with
                           an added column for the a-site location along with
                           data from the annotation file
        target: Target from which the distance is calculated ("start", "stop"
                or "both")
        distance_range: The range checked from the target, both up- and
                        downstream. If no read are found in this range, all
                        reads will be used.

    Outputs:
        metagene_profile_dict: dictionary containing the read_length of
        the read and distance to the target as keys and the counts as values
    """
    annotated_read_df["normalized_counts"] = normalize_counts(annotated_read_df)

    target_loop = [target] if target != "both" else ["start", "stop"]
    metagene_profile_dict = {"start": {}, "stop": {}}
    for current_target in target_loop:
        annotated_read_df["metagene_info"] = metagene_distance(
            annotated_read_df, current_target
        )
        pre_metaprofile_dict = (
            annotated_read_df[
                (annotated_read_df["metagene_info"] > distance_range[0] - 1)
                & (annotated_read_df["metagene_info"] < distance_range[1] + 1)
            ]
            .groupby(["read_length", "metagene_info"])["normalized_counts"]
            .sum()
            .to_dict()
        )
        if pre_metaprofile_dict == {}:
            print(
                "ERR - Metagene Profile: No reads found in specified range, ",
                "removing boundaries..."
            )
            pre_metaprofile_dict = (
                annotated_read_df.groupby(["read_length", "metagene_info"])
                .size()
                .to_dict()
            )
        # Fill empty read length and distance keys with None
        min_length = min([x[0] for x in list(pre_metaprofile_dict.keys())])
        max_length = max([x[0] for x in list(pre_metaprofile_dict.keys())])
        for y in range(min_length, max_length):
            if y not in [x[0] for x in list(pre_metaprofile_dict.keys())]:
                pre_metaprofile_dict[(y, 0)] = 0
            for z in range(distance_range[0], distance_range[1]):
                if z not in [x[1] for x in list(pre_metaprofile_dict.keys())]:
                    pre_metaprofile_dict[(y, z)] = 0

        for key, value in pre_metaprofile_dict.items():
            if key[0] not in metagene_profile_dict[current_target]:
                metagene_profile_dict[current_target][key[0]] = {}
            metagene_profile_dict[current_target][key[0]][key[1]] = value
    return metagene_profile_dict

In [93]:
annotated_read_df["a_site"] = annotated_read_df["reference_start"]

In [105]:
distance_range = [-30,10]
metagene_dict = metagene_profile4(annotated_read_df, "both", distance_range)

In [89]:
previous_window_mean = 0
mean_window_list = {}
for read_length in metagene_dict["start"].keys():
    mean_window_list[read_length] = {}
    for window in range(0,35):
        read_length_window = list(metagene_dict["start"][read_length].values())[window:window+5]
        window_mean = round(sum(read_length_window)/5,3)
        if window != 0:
            mean_difference = round(previous_window_mean - window_mean, 3)
            mean_window_list[read_length][window-30] = mean_difference
        previous_window_mean = window_mean

In [90]:
for read_length in mean_window_list.keys():
    read_length = 28
    fig = go.Figure([go.Bar(x=list(mean_window_list[read_length].keys()), y=list(mean_window_list[read_length].values()))])
    fig.update_layout(
    title=f"{read_length}")
    fig.show()
    break

In [92]:
# def plot_metagene_heatmap(metagene_profile_dict: dict, config: dict) -> dict:
metagene_profile_dict = metagene_dict
"""
Generate a heatmap of the reads depending on their distance
to a target, read length and count

Inputs:
    metagene_heatmap_dict: Dictionary containing the counts as values
        and distance from target as keys
    config: Dictionary containing the configuration information

Outputs:
    plot_metagene_heatmap: Dictionary containing the plot name,
    description and plotly figure for html and pdf export
"""
count = 0
columns = 1
if metagene_profile_dict["start"] != {}:
    target_loop = ["start"]
    if metagene_profile_dict["stop"] != {}:
        target_loop.append("stop")
        columns = 2
else:
    target_loop = ["stop"]

fig = make_subplots(
    rows=1,
    cols=columns,
    shared_yaxes=True,
    subplot_titles=["Distance from 5'", "Distance from 3'"]
    if len(target_loop) > 1
    else ["Distance from 5'"]
    if target_loop == ["start"]
    else ["Distance from 3'"],
)

for current_target in target_loop:
    count += 1
    x_data = []
    y_data = []
    z_data = []
    for k1, v1 in metagene_profile_dict[current_target].items():
        for k2, v2 in v1.items():
            x_data.append(k2)
            y_data.append(k1)
            z_data.append(v2)

    if config["plots"]["metagene_profile"]["max_colorscale"] is None:
        z_data = [z/max(z_data) for z in z_data]

    fig.add_trace(
        go.Heatmap(
            x=x_data,
            y=y_data,
            z=z_data,
            colorscale=config["plots"]["metagene_profile"]["colorscale"],
            zmin=0,
            zmax=config["plots"]["metagene_profile"]["max_colorscale"],
        ),
        row=1,
        col=count,
    )
    fig.update_xaxes(
        title_text="Relative position (nt)",
        row=1,
        col=count,
        title_font=dict(size=18),
        range=config["plots"]["metagene_profile"]["distance_range"]
    )
fig.update_layout(
    title="Metagene Heatmap",
    yaxis_title="Read length",
    font=dict(
        family=config["plots"]["font_family"],
        size=18,
        color=config["plots"]["base_color"],
    ),
    legend={"traceorder": "normal"},
    showlegend=False,
)
if columns > 1:
    fig.update_layout(
        xaxis=dict(
            domain=[0, 0.48], zeroline=False
        ),  # Adjust domain and remove x-axis zeroline for subplot 1
        xaxis2=dict(
            domain=[0.52, 1], zeroline=False
        ),  # Adjust domain and remove x-axis zeroline for subplot 2
    )
else:
    fig.update_layout(
        xaxis=dict(domain=[0, 1], zeroline=False),
    )
fig.show()
    # plot_metagene_heatmap = {
    #     "name": "Metagene Heatmap",
    #     "description": "Metagene heatmap showing the distance between the \
    #         A-site and a target per read length and the counts in colorscale.",
    #     "fig_html": pio.to_html(fig, full_html=False),
    #     "fig_image": plotly_to_image(fig,
    #                                  config["plots"]["image_size"][0],
    #                                  config["plots"]["image_size"][1]),
    # }
    # return plot_metagene_heatmap

AAAAGGGTTC
AAAAGGGTTC
CTAAAGGTTC
AAAAGGGTTCA
AAAAGGGTTCA
GGTCCCCAAA
GGTCCCCAAA
GGTCCCCAAA

```
██████╗  ██╗ ██████╗  ██████╗ ███████╗ ██████╗ ███╗   ███╗███████╗
██╔══██╗ ██║ ██╔══██╗██╔═══██╗██╔════╝██╔═══██╗████╗ ████║██╔════╝
██████╔╝ ██║ ██████╔╝██║   ██║███████╗██║   ██║██╔████╔██║█████╗
██╔══██╗ ██║ ██╔══██╗██║   ██║╚════██║██║   ██║██║╚██╔╝██║██╔══╝
██║  ██║ ██║ ██████╔╝╚██████╔╝███████║╚██████╔╝██║ ╚═╝ ██║███████╗
╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝ ╚══════╝ ╚═════╝ ╚═╝     ╚═╝╚══════╝

   ███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗   
   ████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝   
   ██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗   
   ██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║   
   ██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║   
   ╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝   

██████╗  ██╗ ██████╗  ██████╗  ███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗  
██╔══██╗ ██║ ██╔══██╗██╔═══██╗ ████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝  
██████╔╝ ██║ ██████╔╝██║   ██║ ██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗   
██╔══██╗ ██║ ██╔══██╗██║   ██║ ██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║
██║  ██║ ██║ ██████╔╝╚██████╔╝ ██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║
╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝  ╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝   
```

In [165]:
from rich.console import Console
from rich.text import Text
logo = Text(
    """
██████╗  ██╗ ██████╗  ██████╗ 
██╔══██╗ ██║ ██╔══██╗██╔═══██╗
██████╔╝ ██║ ██████╔╝██║   ██║
██╔══██╗ ██║ ██╔══██╗██║   ██║
██║  ██║ ██║ ██████╔╝╚██████╔╝
╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝ 
""",
    style="bold blue",
)
logo += Text(
    """
███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗   
████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝   
██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗   
██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║   
██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║   
╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝   
""",
    style="bold red",
)
# LINE 1
logo = Text(
    """
    ██████╗  ██╗ ██████╗  ██████╗  """,
    style="bold blue",end=""
)
logo += Text(
    """███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗ """,
    style="bold red",end=""
)
# LINE 2
logo += Text(
    """
    ██╔══██╗ ██║ ██╔══██╗██╔═══██╗ """,
    style="bold blue",
)
logo += Text(
    """████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝""",
    style="bold red",end=""
)
# LINE 3
logo += Text(
    """
    ██████╔╝ ██║ ██████╔╝██║   ██║ """,
    style="bold blue",
)
logo += Text(
    """██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗""",
    style="bold red",end=""
)
# LINE 4
logo += Text(
    """
    ██╔══██╗ ██║ ██╔══██╗██║   ██║ """,
    style="bold blue",
)
logo += Text(
    """██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║""",
    style="bold red",end=""
)
# LINE 5
logo += Text(
    """
    ██║  ██║ ██║ ██████╔╝╚██████╔╝ """,
    style="bold blue",
)
logo += Text(
    """██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║""",
    style="bold red",end=""
)
# LINE 6
logo += Text(
    """
    ╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝  """,
    style="bold blue",
)
logo += Text(
    """╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝""",
    style="bold red",end=""
)
Console().print(logo)

    ██████╗  ██╗ ██████╗  ██████╗  ███╗   ███╗███████╗█████████╗██████╗  ██╗  ██████╗  ███████╗ 
    ██╔══██╗ ██║ ██╔══██╗██╔═══██╗ ████╗ ████║██╔════╝╚══██╔═══╝██╔══██╗ ██║ ██╔════╝ ██╔═════╝
    ██████╔╝ ██║ ██████╔╝██║   ██║ ██╔████╔██║█████╗     ██║    ██████╔╝ ██║ ██║      ████████╗
    ██╔══██╗ ██║ ██╔══██╗██║   ██║ ██║╚██╔╝██║██╔══╝     ██║    ██╔══██╗ ██║ ██║      ╚═════██║
    ██║  ██║ ██║ ██████╔╝╚██████╔╝ ██║ ╚═╝ ██║███████╗   ██║    ██║  ██║ ██║ ╚██████╗ ████████║
    ╚═╝  ╚═╝ ╚═╝ ══════╝  ╚═════╝  ╚═╝     ╚═╝╚══════╝   ╚═╝    ╚═╝  ╚═╝ ╚═╝  ╚═════╝ ╚═══════╝